<a href="https://colab.research.google.com/github/isb-cgc/Community-Notebooks/blob/master/HTAN/Python%20Notebooks/Analyzing_HTAN_spatial_data_with_BigQuery_geospatial_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing HTAN spatial data with BigQuery spatial analytics

        Title:   Analyzing HTAN spatial data with BigQuery geospatial analytics
        Author:  Boris Aguilar (baguilar@systemsbiology.org) & Vesteinn Thorsson
        Created: 02/14/2024
        Purpose: Provide examples of how BigQuery geospatial analytics can be used to analyze spatial data in HTAN.
        URL:     NA
        Notes:   NA

# 1. Introduction & Overview
[HTAN](https://humantumoratlas.org/) is a National Cancer Institute (NCI)-funded Cancer Moonshot initiative to construct 3-dimensional atlases of the dynamic cellular, morphological, and molecular features of human cancers as they evolve from precancerous lesions to advanced disease [[Cell April 2020](https://www.sciencedirect.com/science/article/pii/S0092867420303469)].


Many HTAN Research Centers employ highly-multiplexed imaging to gain understanding of molecular processes and interactions at work in the tumor microenvironment.  The study [The Spatial Landscape of Progression and Immunoediting in Primary Melanoma at Single-Cell Resolution](https://aacrjournals.org/cancerdiscovery/article/12/6/1518/699151/The-Spatial-Landscape-of-Progression-and), Ajit Nirmal et al, Cancer Discov (2022) 12 (6): 1518–1541, uses multiplexed whole-slide imaging analysis to characterize intermixed and graded morphological and molecular features in human melanoma cancer samples, highlighting large-scale cancer-characteristic structural features. Spatial profiling of proteins and mRNA reveals recurrent morphologic and molecular features of tumor evolution that involve localized paracrine cytokine signaling and direct cell–cell contact.

[Google BigQuery Geospatial Analytics](https://cloud.google.com/bigquery/docs/geospatial-intro) enables seamless and efficient analysis and visualization of geospatial data within Google BigQuery. 

### 1.1 Goal

This notebook offers examples demonstrating how BigQuery spatial analytics can assist in the analysis and exploration of spatial data accessible within HTAN BigQuery tables.

### 1.2 Inputs, Outputs, & Data

This notebook utilizes spatial transcriptomics data from Melanoma samples within the ISB-CGC HTAN dataset (https://isb-cgc.appspot.com/). Specifically, we will be utilizing the public BigQuery table named [isb-cgc-bq.HTAN.imaging_level4_HMS_mel_mask_current](https://console.cloud.google.com/bigquery?p=isb-cgc-bq&d=HTAN&t=imaging_level4_HMS_mel_mask_current&page=table). The originating data can be found on the [HTAN Data Portal](https://data.humantumoratlas.org/).




# 2. Environment and Module setup

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
import pandas_gbq
#from google.colab import auth ## Can be omitted if you are running locally

# 3. Google Authentication
Running the BigQuery cells in this notebook requires a Google Cloud Project, instructions for creating a project can be found in the [Google Documentation](https://cloud.google.com/resource-manager/docs/creating-managing-projects#console). The instance needs to be authorized to bill the project for queries. For more information on getting started in the cloud see [Quick Start Guide to ISB-CGC](https://nbviewer.org/github/isb-cgc/Community-Notebooks/blob/master/Notebooks/Quick_Start_Guide_to_ISB_CGC.ipynb) and alternative authentication methods can be found in the [Google Documentation](https://cloud.google.com/resource-manager/docs/creating-managing-projects#console).

In [2]:
# if you're using Google Colab, authenticate to gcloud with the following:
auth.authenticate_user()

# alternatively, use the gcloud SDK
# !gcloud auth application-default login

NameError: name 'auth' is not defined

In [ ]:
# For running in this teaching session we provided a service account authentication
%env GOOGLE_APPLICATION_CREDENTIALS=/home/jovyan/key-file

# set the google project that will be referenced for this notebook's computations
project_id = 'isb-cgc-outreach'

# Create a client to access the data within BigQuery
client = bigquery.Client(project_id)


# 4. Exploration and analysis of HTAN spatial data with BigQuery

We will focus on imaging data of sample MEL1-1 as described in [the manuscript](https://aacrjournals.org/cancerdiscovery/article/12/6/1518/699151/The-Spatial-Landscape-of-Progression-and). This tissue section corresponds to HTAN biospecimen `HTA7_1_3`. An excellent [interactive guide](https://labsyspharm.github.io/HTA-MELATLAS-1/stories/MEL1-full-story.html
) to the to multiplex imaging data for this biospecimen is available as [Minerva Story](https://github.com/labsyspharm/minerva-story/wiki).

The necessary data can be found in the BigQuery table [isb-cgc-bq.HTAN.imaging_level4_HMS_mel_mask_current](https://console.cloud.google.com/bigquery?p=isb-cgc-bq&d=HTAN&t=imaging_level4_HMS_mel_mask_current&page=table). This table contains estimated marker intensities following cell segmentation, along with data on the location of segmented cells. For our analysis, we will utilize the cell locations denoted by the centroids X_centroid and Y_centroid, representing the coordinates of cells in pixels (where 1 pixel corresponds to 0.65 micrometers).


## 4.1 Labeling tumor cells
Firstly, we will label cancer cells within the samples. As per the manuscript, cancer cells typically exhibit positive expression of SOX10, along with  positive S100B and/or positive CD63. This labeling process involves applying a predefined thresholds for the expression levels of these markers. We calibrated these tresholds manually; but the manuscript provides more details about how to identify cell types from markers.

Finally we will focus on a spatial region of HTA7_1_3  labeled  Invasive Melanoma (IM), delineated by the following corner coordinates: (23076.9, 9615.3) and (30384.6, 15000).

The follow query is an example of how to label tumor cells using the required markers which are in the columns  `S100B_cellRingMask`, `S100B_cellRingMask`, and `CD63_cellRingMask` :

In [ ]:
query = """
WITH cells AS (
  SELECT  CellID, X_centroid,	Y_centroid,
    IF ( SOX10_cellRingMask > 3704.5 AND (S100B_cellRingMask > 7589.48 OR CD63_cellRingMask > 570.68 ),
      'Tumor', 'Other'  ) AS celltype
  FROM `isb-cgc-bq.HTAN.imaging_level4_HMS_mel_mask_current`
  WHERE HTAN_Biospecimen_ID = 'HTA7_1_3'
)
SELECT CellID, X_centroid,	Y_centroid, celltype
FROM cells
WHERE X_centroid > 23076.9 AND X_centroid < 30384.6
  AND Y_centroid > 9615.3 AND Y_centroid < 15000
"""
df = pandas_gbq.read_gbq(query,project_id=project_id)
df

The following code generates an image of the location of Tumor and Other cells from the above dataframe. This image is similar to Figure 3A of the [manuscript](https://aacrjournals.org/cancerdiscovery/article/12/6/1518/699151/The-Spatial-Landscape-of-Progression-and).  

In [ ]:
# Set the seaborn theme
sns.set_style("white")

# Let's plot the centroids and color them by marker first:
g=sns.scatterplot(y="Y_centroid", x="X_centroid", hue='celltype', s=5,  data=df, alpha = 0.5, palette="icefire")

sns.move_legend(g, "upper right", title='Cell type',  bbox_to_anchor=(1.2, 1.01))
plt.axis('equal')
g.invert_yaxis() # Flip the y-axis so it aligns with the figure in the publication

## 4.2 Computing local neighborhoods using BigQuery Geospatial analytics

Often, spatial analysis necessitates the identification of neighboring cells, which are cell pairs situated closer than a specified threshold distance. Here, we employ BigQuery spatial analytics to identify neighboring cells for each tumor cell.

BigQuery operates with distances in spherical geographic coordinate, thus requiring the rescaling of pixel coordinates to latitude and longitude. We found that a good approximation is to assume that the maximum dimension of the image in pixels corresponds to 0.1 Earth angular degrees. Given that the HTA7_1_3 image dimensions are (36857, 25808) pixels, we rescale the cell coordinates by a factor of 1/368570. Furthermore, based on this assumption, 20 micrometers equates to 9.29324770787722 meters, serving as the threshold distance in BigQuery.
  
The following query initially labels all cells in HTA7_1_3 as either "Tumor" or "Other" and creates a [ST_GeogPoint](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_geogpoint) object defining a geospatial point in BigQuery. Subsequently, the query utilizes the [ST_DWithin](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_within) function to generate a table in which rows correspond to pairs of cells that are closer than 9.29324770787722 meters.

In [ ]:
query = """
WITH geodat AS(
   SELECT CellID, X_centroid, Y_centroid,
    IF ( SOX10_cellRingMask > 3704.5 AND (S100B_cellRingMask > 7589.48 OR CD63_cellRingMask > 570.68 ),
      'Tumor', 'Other'  ) AS celltype,
    ST_GeogPoint(X_centroid/368570 ,Y_centroid/368570 ) as p,
   FROM `isb-cgc-bq.HTAN.imaging_level4_HMS_mel_mask_current`
   WHERE HTAN_Biospecimen_ID='HTA7_1_3'
)
SELECT t1.CellID , t1.X_centroid, t1.Y_centroid, t1.p, t1.celltype,
  t2.CellID AS  CellID_1, t2.X_centroid AS X_centroid_1, t2.Y_centroid AS Y_centroid_1, t2.p AS p_1, t2.celltype AS celltype_1,
  ST_Distance(t1.p,t2.p) as Distance
FROM geodat as t1
JOIN geodat as t2
ON ST_DWithin(t1.p, t2.p, 9.29324770787722)
"""

It is important to highlight that HTA7_1_3 comprises over 1.1 million cells, and the output of the query mentioned above yields a table with nearly 14 million rows. Therefore, rather than downloading the data, we opt to generate a new BigQuery table containing the query output. The BigQuery table will be generated in the Google Cloud Project you specified as `project_id` above. Should the table already exist, the code will simply overwrite it.


In [ ]:
# create a dataset with name temp15432
dataset_id = dataset = bigquery.Dataset( "{}.temp15432".format(client.project) )
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok = True, timeout=30)

# Set destination table options (optional)
#destination_table = "{}.temp15432.Melanoma_CyCIF_HTA7_1_3_points_within_20um".format(client.project)  # Adjust to your desired location

#job_config = bigquery.QueryJobConfig(destination=destination_table, write_disposition='WRITE_TRUNCATE' )

query_job = client.query(query)  # Make an API request.
query_job.result()  # Wait for the job to complete

We print the first 10 rows of the generated table. Each row represents a pair of cells with a distance within 20 micrometers (corresponding to 9.29324770787722 earth meters after scaling)

In [ ]:
query = """SELECT *
FROM {}.temp15432.Melanoma_CyCIF_HTA7_1_3_points_within_20um LIMIT 10
""".format( client.project  )
df = pandas_gbq.read_gbq(query,project_id=project_id)
df

## 4.3 Distribution of neighboring tumor cells
We can utilize the table of neighboring cells to determine the distribution of neighboring tumor cells for each tumor cell, i.e., the number of tumor cells located in the vicinity of each tumor cell.

In [ ]:
query = """
WITH cellp AS (
  SELECT CellID, celltype, CellID_1, celltype_1
  FROM `{}.temp15432.Melanoma_CyCIF_HTA7_1_3_points_within_20um`
  WHERE  X_centroid > 23076.9 AND X_centroid < 30384.6
  AND Y_centroid > 9615.3 AND Y_centroid < 15000

)
SELECT CellID, COUNTIF(celltype_1 = 'Tumor') -1  AS N_Tumor_Cells,
FROM cellp
WHERE celltype = 'Tumor'
GROUP BY CellID
""".format( client.project  )
df1 = pandas_gbq.read_gbq(query,project_id=project_id)


In [ ]:
sns.histplot(data=df1, x="N_Tumor_Cells")


It appears that most of the tumor cells in this Invasive Melanoma region are surrounded by approximately 10 other tumor cells

## 4.4 Clustering tumor cells



BigQuery Geospatial analytics provides additional functions for analyzing spatial data. Here, we utilized the [ST_CLUSTERDBSCAN](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_clusterdbscan) function to identify spatial clusters of tumor cells. This function implements [the DBSCAN algorithm](https://en.wikipedia.org/wiki/DBSCAN), which is capable of identifying high-density clusters of data.

In [ ]:
query = """
WITH cells AS (
    SELECT CellID, X_centroid, Y_centroid,
      ST_GeogPoint(X_centroid/368570 ,Y_centroid/368570 ) as p
    FROM `isb-cgc-bq.HTAN.imaging_level4_HMS_mel_mask_current`
    WHERE HTAN_Biospecimen_ID='HTA7_1_3'
      AND X_centroid > 23076.9 AND X_centroid < 30384.6
      AND Y_centroid > 9615.3 AND Y_centroid < 15000
      AND SOX10_cellRingMask > 3704.5
      AND (S100B_cellRingMask > 7589.48 OR CD63_cellRingMask > 570.68 )
)
SELECT CellID, X_centroid, Y_centroid, ST_CLUSTERDBSCAN(p, 20, 10) OVER () AS cluster_num
FROM cells
ORDER BY cluster_num
"""
df = pandas_gbq.read_gbq(query,project_id=project_id)
df

The following code generates an image depicting the locations of tumor cells, with each cell colored according to the cluster to which it belongs.

In [ ]:
df1 = df.dropna()
df1['cluster_num'] = df1['cluster_num'].astype(  str )

# Plot it
sns.set_style("white")
g=sns.scatterplot(data=df1, y="Y_centroid", x="X_centroid", hue='cluster_num', s=5,   alpha = 0.5 , palette="icefire")
g.invert_yaxis()

# 5. Relevant links and citations

*   [Link to the Manuscript](https://aacrjournals.org/cancerdiscovery/article/12/6/1518/699151/The-Spatial-Landscape-of-Progression-and)
*   [Minerva Link showing an image of HTA7_1_3](https://labsyspharm.github.io/HTA-MELATLAS-1/stories/MEL1-full-story.html)
*   [Link with the functions available in BigQuery geospatial analyticis](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions)

